In [8]:
# Cross validation

In [5]:
import sys
sys.path.append('..')

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
from utils import utils

In [7]:
nh3_nc = xr.load_dataset('../datasets/CAMS-GLOB-ANT_Glb_0.1x0.1_anthro_nh3_v4.2_monthly_lombardia.nc') # Copernicus (0.1°x0.1°)
nh3_nc.agl

<xarray.DataArray 'agl' (time: 248, lat: 19, lon: 28)>
array([[[           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan, 1.40527645e-05,            nan],
        ...,
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan]],

       [[           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan, 1.54474892e-05,            nan],
...
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan]],

       [[           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan, 3.44685031e-05,            nan],
        ...,
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan]]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-15 2000-02-15 ... 2020-08-15
  * lat      (lat) float64 44.75 44.85 44.95 45.05 ... 46.25 46.35 46.45 46.55
  * lon      (lon) float64 8.55 8.65 8.75 8.85 8.95 ... 10.95 11.05 11.15 11.25

In [14]:
# MAX: lat(45.35 6) lon(10.25 17)
# TIME: '2000-08-15' '2019-08-15' (Training set)
# Scarto i primi mesi fino ad settembre del 2000 per avere esattamente 19 anni, tenendo l'ultimo anno per il training
# Abbiamo 228 osservazioni mensili, con k scegliamo quante usarle pere il training, la restante parte verrà usata per il test (Arrotondiamo per anno magari)
k = 204
nh3_max = utils.cross_validation(nh3_nc.agl, 6, 17, '2000-09-15', '2019-08-15', k)

SyntaxError: invalid syntax (<ipython-input-14-4f51bad478a3>, line 1)

In [32]:
# TODO RIMUOVERE

import pandas as pd
import numpy as np
import sklearn.linear_model as skl_lm
import matplotlib.pyplot as plt

lat = 6
lon = 17
data = nh3_nc.agl[:,lat,lon].to_series()
start_time = '2000-09-15'
end_time = '2019-08-15'
k = 204

data = data[start_time:end_time]

data

time
2000-08-15    0.000231
2000-09-15    0.000223
2000-10-15    0.000183
2000-11-15    0.000119
2000-12-15    0.000091
                ...   
2019-04-15    0.000148
2019-05-15    0.000208
2019-06-15    0.000244
2019-07-15    0.000284
2019-08-15    0.000233
Name: agl, Length: 229, dtype: float64

In [ ]:
# Converting xarray to pandas and select the coordinates
data = utils.xarray2pandas(data, lon, lat)

# Selecting original training dataset
data = data['2000-08-15':'2019-08-15']


# We begin by using the sample() function to split the set of observations into two halves: training and test dataset
# k is the amount of observation used for the test
# random_state is a seed for python's random number generator, so that you'll obtain precisely the same results each time. It is generally a good idea to set a random seed when performing an analysis such as cross-validation that contains an element of randomness, so that the results obtained can be reproduced precisely at a later time.
train_df = data.sample(k, random_state = 1)
test_df = data[~data.isin(train_df)].dropna(how = 'all')